In [1]:
from decoder_head.core import *
from fastai2.text.all import *

In [2]:
vocab_en= make_vocab(pd.read_pickle('data/en-100_tok/counter.pkl'), max_vocab=4000)
vocab_es= make_vocab(pd.read_pickle('data/es-100_tok/counter.pkl'), max_vocab=4000)

In [3]:
path = 'data/en-100_tok/'
mult = 4
bs = 80
seq_len = 70

lm = DataBlock(blocks=(TextBlock(vocab=vocab_en, is_lm=True),),
                get_x=read_tokenized_file,
                get_items=partial(get_text_files, folders=['train', 'valid']),
                splitter=FuncSplitter(lambda itm: itm.parent.name == 'valid'))

dbunch_lm = lm.databunch(path, path=path, bs=bs, seq_len=seq_len)

In [4]:
learn = language_model_learner(
    dbunch_lm,
    pAWD_LSTM,
    opt_func=opt,
    pretrained=False,
    config=awd_lstm_lm_config,
    drop_mult=0.1,
    metrics=[accuracy, top_k_accuracy]
)

In [5]:
# learn.load('pLSTM_en')

In [6]:
with open('data/en-es.txt') as f:
    en_es = f.readlines()
    
en_es = [l.strip() for l in en_es]

In [7]:
en_es_dict = defaultdict(list)

for l in en_es:
    source, target = l.split()
    en_es_dict[source].append(target)

In [8]:
len(en_es_dict)

93084

Check that we have the source word in the model trained on English

In [9]:
vocab_en_set = set(vocab_en)
en_es_dict = {k: v for k, v in en_es_dict.items() if k in vocab_en_set}

In [10]:
len(en_es_dict)

3475

Make sure we have the target word in Spanish

In [11]:
vocab_es_set = set(vocab_es)
en_es_dict = {k: vocab_es_set.intersection(set(v)) for k, v in en_es_dict.items() if vocab_es_set.intersection(set(v))}

In [12]:
len(en_es_dict)

2451

Attemp translation from English to Spanish

In [13]:
learn.load('pLSTM_en')

In [14]:
learn.model[0].encoder.weight.requires_grad

True

In [15]:
learn.optimize_permutation()

In [16]:
learn.model[0].encoder.weight.requires_grad

False

In [17]:
nn.init.kaiming_normal_(learn.model[0].encoder.p);

In [18]:
learn.model[0].encoder.weight.data = torch.load('data/embeddings_es.torch').data

In [19]:
learn.model[0].encoder.p.requires_grad

True

In [20]:
learn.model[0].encoder.weight.requires_grad

False

We now have an English LM with Spanish embeddings loaded. Let's see how we do on translation.

One thing worth keeping in mind are the missing English words from the Spanish vocabulary. This is an interesting situation and makes the task even harder.

In [21]:
learn.loss_func = lambda preds, targs: aza_loss(learn, preds, targs)

In [22]:
learn.fit_one_cycle(1, 1e-3,  moms=(0.8, 0.7, 0.8), wd=1e-7)

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,4.984266,4.949764,0.250956,0.460905,41:12


In [25]:
# learn.save('pLSTM_en-es')

In [30]:
learn.load('pLSTM_en-es')

In [31]:
learn.validate()

(#3) [4.975531101226807,0.24766667187213898,0.45390090346336365]

In [32]:
def top_n_translation_acc(top_n=1):
    p = learn.model[0].encoder.p
    hits = 0
    for k, v in en_es_dict.items():
        idx_en = vocab_en.index(k)
        for vv in v:
            idx_es = vocab_es.index(vv)
            if idx_es in p[idx_en].argsort(descending=True)[:top_n]:
                hits += 1
#                 print(k, vv)
                break
    return hits/len(en_es_dict)

In [33]:
top_n_translation_acc(100)

0.037943696450428395

In [34]:
def top_n_translation_acc(top_n=1):
    p = learn.model[0].encoder.p
    hits = 0
    for k, v in en_es_dict.items():
        idx_en = vocab_en.index(k)
        for vv in v:
            idx_es = vocab_es.index(vv)
            if idx_es in p[idx_en].argsort(descending=True)[:top_n]:
                hits += 1
                print(k, [vocab_es[i] for i in p[idx_en].argsort(descending=True)[:5]])
                break
    return hits/len(en_es_dict)

In [35]:
en_es_dict['have']

{'han', 'tener', 'tienen'}

In [36]:
top_n_translation_acc(5)

has ['utilizan', 'deben', 'símbolo', 'permiten', 'tiene']
one ['cuando', 'para', 'lo', 'una', 'algo']
united ['evitar', 'animales', 'tratar', 'fuego', 'unidos']
spanish ['juvenil', 'high', 'masculino', 'española', 'blancas']
publishing ['núcleos', 'documentos', 'diez', 'publicaciones', 'satélite']


0.002039983680130559

In [23]:
def top_n_translation_acc(top_n=1):
    p = learn.model[0].encoder.p
    hits = 0
    for k, v in en_es_dict.items():
        idx_en = vocab_en.index(k)
        print(k, [vocab_es[i] for i in p[idx_en].argsort(descending=True)[:5]])
        for vv in v:
            idx_es = vocab_es.index(vv)
            if idx_es in p[idx_en].argsort(descending=True)[:top_n]:
                hits += 1
                break
    return hits/len(en_es_dict)

In [24]:
top_n_translation_acc(100)

the ['argumento', 'raza', 'lo', 'poblacional', 'estudio']
and ['3', 'batalla', 'juego', 'donde', 'mientras']
was ['debido', 'un', 'local', 'que', 'de']
for ['de', 'guerra', 'poblacional', 'real', 'ley']
that ['si', 'ni', 'mientras', 'nada', 'cuando']
with ['corte', 'otro', 'central', 'real', 'oficina']
from ['oficina', 'de', 'the', 'real', 'condado']
this ['su', 'oficial', 'debido', 'según', 'internacional']
his ['incluyendo', 'teniendo', 'siendo', 'obteniendo', 'aunque']
not ['basado', 'incluyendo', 'como', 'utilizando', 'usando']
are ['teniendo', 'local', 'mediterráneo', 'punto', 'cuya']
talk ['sonidos', 'apple', 'la', 'ciencias', 'israel']
which ['es', 'de', 'dado', 'densidad', 'urbano']
also ['vez', 'junto', 'local', 'oficial', 'característica']
has ['geografía', 'puesta', 'basada', 'capaz', 'descubrimiento']
were ['incluyendo', 'teniendo', 'local', 'diciendo', 'cuya']
but ['es', 'pero', 'media', 'veces', 'aunque']
have ['capaz', 'utilizan', 'basada', 'puesta', 'descubrimiento']
on

local ['trata', 'compone', 'ubica', 'amigos', 'orillas']
around ['fin', 'especial', 'del', 'poblacional', 'frente']
river ['sonora', 'alternativa', 'respecto', 'prensa', 'crítica']
top ['pues', 'tales', 'tal', 'creando', 'interés']
main ['trata', 'debe', 'tiene', 'local', 'medio']
language ['popular', 'largo', 'film', 'democrática', '2011']
french ['motivo', '&', '«', 'local', 'representantes']
named ['ángel', 'claro', 'orden', 'domingo', 'les']
note ['gobernador', 'amarillo', 'graves', 'niña', 'producciones']
career ['solista', 'televisión', 'hasta', 'internet', 'invitado']
original ['debe', 'le', 'europa', 'trata', 'medio']
age ['rico', 'bautista', 'unido', 'para', 'cerca']
service ['mexicano', 'solista', 'hasta', 'toda', 'todo']
established ['por', 'ni', 'e', 'donde', '…']
located ['nº', 'grados', 's.', 'p.', 'década']
said ['papel', 'alto', 'puesto', 'porque', 'pero']
website ['single', 'tony', 'vídeo', 'disco', 'mike']
population ['josé', 'garcía', 'lópez', 'evitar', 'gracias']
ai

battle ['manera', 'vivo', 'parte', 'cerca', 'hacer']
sports ['guinea', 'vía', 'europa', 'colombia', 'sudamérica']
already ['utilizado', 'basada', 'usando', 'científica', 'usa']
currently ['entrevista', 'china', 'basados', 'defensor', 'físicas']
hall ['tenía', 'parte', 'en', 'era', 'fue']
buildings ['distribución', 'toda', 'cine', 'todo', 'televisión']
historic ['maria', 'podemos', 'joan', 'teresa', 'tomás']
date ['contemporáneo', 'mexicano', 'responsable', 'latina', 'invitado']
considered ['y', 'e', 'otro', 'u', 'ella']
change ['mundial', 'mexicano', 'sus', 'europeo', 'rock']
location ['blanco', 'marrón', 'colón', 'gas', 'negro']
seems ['primer', '30', 'propósito', 'primero', 'principio']
must ['rango', 'profesor', 'cual', 'grado', 'maestro']
our ['dada', 'causa', 'aprobó', 'mediante', 'determinado']
southern ['cristo', 'maría', 'virgen', 'les', 'decir']
lost ['qué', 'esa', 'su', 'realidad', 'fin']
something ['don', 'y', 'medio', 'al', 'obteniendo']
together ['mejor', 'sonora', 'alcanz

border ['lópez', 'garcía', 'actuar', 'producir', 'tomar']
medical ['inaugurado', 'realizados', 'asturias', 'realiza', 'vía']
countries ['televisión', 'música', 'cine', 'radio', 'producir']
past ['american', 'universitario', 'naciones', 'largo', 'united']
writing ['and', 'y', 'por', 'of', 'in']
makes ['e', 'u', 'ni', 'o', ';']
interest ['vida', 'ventas', 'inicios', 'rock', 'álbumes']
killed ['vida', 'esa', '2018', '2014', '2017']
medal ['aparecer', 'manera', 'fácilmente', 'papeles', 'numerosas']
signed ['and', 'será', 'persona', 'donde', 'que']
fair ['empresas', 'pasar', '1975', 'tareas', 'centro']
search ['y', 'altamente', 'parcialmente', 'totalmente', 'públicos']
bay ['2015', 'fuera', '2018', '2016', '2019']
reference ['tiempos', 'áfrica', 'finales', 'óscar', 'nuestros']
especially ['puestos', 'realizados', 'pobres', 'días', 'año']
library ['distribución', 'población', 'proyectos', 'economía', 'evolución']
management ['1909', '1946', '1979', 'propuso', 'plena']
references ['1915', 'áf

meet ['costa', 'isla', 'factor', 'indígena', 'masa']
box ['unió', 'utilizar', 'american', 'utilizando', 'parís']
lines ['crear', 'cine', 'grabar', 'mercado', 'todo']
democratic ['hacienda', 'importantes', 'destacar', 'ramón', 'llamadas']
success ['porque', 'ese', 'esto', 'colón', 'pop']
associated ['formar', 'materia', 'castilla', '2016', '2017']
singles ['esta', 'cómo', 'plantas', 'art', 'europa']
traditional ['guillermo', 'trata', 'partes', 'debe', 'cuentan']
rest ['grande', 'bautista', 'peligro', 'ahí', 'do']
highway ['desde', 'entre', 'todos', 'internacional', '..']
matter ['construir', 'comenzó', 'desarrollar', 'mostrar', 'empezó']
particularly ['conocida', 'realizados', 'argentina', 'económicas', 'realizada']
wide ['acompañado', 'volver', 'fecha', 'unirse', 'favor']
month ['realmente', 'oficialmente', 'cómo', 'español', 'principal']
care ['pues', 'habiendo', 'justo', 'siguiendo', 'especialmente']
cultural ['brasil', 'italia', 'asturias', 'galicia', 'pasar']
commission ['recibió',

...

americans ['varios', 'antecedentes', 'recibió', 'llamó', 'respecto']
introduction ['he', 'hay', 'gris', 'hecho', 'encontrado']
vehicle ['produjo', 'regional', 'ha', 'comenzó', 'realizó']
stock ['bilbao', 'joseph', 'escrito', 'servir', 'pieza']
vehicles ['film', 'respuesta', 'regular', 'drama', 'arte']
frequently ['diez', 'jugando', 'profesional', 'frente', 'igualmente']
performing ['provenientes', 'carga', 'biología', 'transición', 'industrias']
northwest ['realizado', 'villa', 'lópez', '2015', 'encontrar']
visit ['der', 'von', 'i', 'morir', '1947']
residence ['procedentes', 'asociados', '1916', 'espacios', 'cuidado']
walter ['dan', 'teresa', 'principales', 'hacen', 'realizadas']
moon ['respecto', 'fueron', 'tuvo', 'colaboró', 'original']
limit ['histórico', 'casi', 'dieron', 'notable', 'necesidades']
minute ['radio', 'latina', 'ciudad', 'teatral', 'conocido']
reduced ['alemán', 'argentino', 'millón', 'francés', 'español']
wind ['después', 'tras', 'característica', 'gracias', 'útil']
o

twelve ['aproximadamente', 'diferentes', 'unos', 'tres', 'varios']
universal ['emilio', 'galicia', 'notables', 'terminal', 'permitir']
skills ['matar', 'ingeniero', 'destino', 'noticias', 'comenzó']
passenger ['genérico', 'científicos', 'usos', 'usa', 'producto']
agriculture ['actitud', 'raíces', 'separación', 'propietario', '60']
decades ['jazz', 'inmediato', 'música', 'importancia', 'usar']
receiving ['efectos', 'europea', 'previamente', 'imperial', 'costa']
signal ['hija', 'logrando', 'empleados', 'perspectiva', 'jack']
reform ['renta', 'valenciana', 'popular', 'murcia', 'niña']
organisation ['republicano', 'mente', 'love', 'catalán', 'tono']
column ['international', 'habla', 'inversión', 'soberanía', 'cambió']
soul ['primo', 'adultos', 'individuos', 'terror', 'plantas']
entrance ['musicales', 'vasco', 'áfrica', 'comienzos', 'tuvo']
path ['moderno', 'completamente', 'valor', 'femenino', 'frecuente']
string ['próximo', 'jack', 'frank', 'visión', 'suecia']
humans ['film', 'verdadero',

buy ['religión', 'aplicaciones', 'tuvieron', 'aparentemente', 'apariencia']
cavalry ['joseph', 'castilla', 'tecnología', 'química', 'cultural']
allied ['motivo', 'características', 'representantes', 'personajes', 'eventos']
involvement ['norte', '200', '2019', 'altas', 'ua']
anime ['existen', 'propuesta', 'usualmente', 'planeta', 'localizado']
sisters ['volvió', 'estatus', 'visto', 'novelas', 'post']
impossible ['marco', 'momento', 'públicos', 'tiempos', 'castellano']
cook ['sector', '1991', 'territorio', 'cambia', '1961']
drawing ['brazo', 'ocupó', 'debería', 'louis', 'habla']
moves ['conde', '1956', 'aquella', '1982', '1966']
ownership ['1983', 'abajo', '1922', 'específicamente', 'valley']
crash ['asociados', 'brian', '1904', 'famosa', 'ben']
guitarist ['indica', 'encontraron', 'navidad', 'emilio', 'oscar']
tone ['igualmente', 'asumió', 'fondo', '2007', 'empleados']
personality ['hacía', 'líder', 'tony', 'considerados', 'video']
reception ['solista', 'municipal', 'porque', ';', 'invi

difficulty ['cola', '1980', 'raúl', 'orientación', 'renunció']
eggs ['masculino', 'femenino', 'crear', 'próximo', 'representar']
enemies ['adultos', 'contemporáneo', 'televisión', 'aumentar', 'música']
cult ['verdadera', 'xxwrep', '1928', 'largas', 'norma']
recognised ['recursos', 'comienza', 'cabe', 'viejo', 'creció']
preparation ['ha', 'generalmente', 'directamente', 'reconocida', 'quedó']
targets ['max', 'raíces', 'tomado', 'aspecto', 'tono']
alien ['colecciones', 'extendió', 'casó', 'escenario', 'publicaciones']
withdrew ['asimismo', 'aun', 'detrás', 'oficialmente', 'anteriormente']
depicted ['externos', 'azul', 'brown', 'inmediatamente', 'año']
thirteen ['vieron', 'causas', 'xxfake', 'ciencias', 'solamente']
measured ['papeles', '1906', 'libro', 'destacados', 'famoso']
flights ['vieron', 'xxfake', 'instrumentos', 'clásicos', 'ácido']
temperatures ['p.', 'notables', 'comedia', 'primo', 'patrimonio']
acres ['mismo', 'rango', 'primero', 'uso', 'nombre']
coins ['buena', 'viejo', 'aun'

0.04528763769889841

In [38]:
top_n_translation_acc(100)

the ['respecto', 'raza', 'poblacional', 'sociedad', 'base']
and ['poblacional', 'humano', 'económico', 'unidos', 'formula']
was ['local', 'cuya', 'que', 'formula', 'de']
for ['de', 'oficina', 'real', 'ley', 'estados']
that ['mientras', 'de', ';', 'real', 'qué']
with ['corte', 'oficina', 'iglesia', 'central', 'policía']
from ['estación', 'oficina', 'the', 'condado', 'secretaría']
this ['basado', 'científico', 'debe', 'acerca', 'debido']
his ['como', 'junto', 'lista', 'medalla', 'x']
not ['basado', 'encargado', 'diseñado', 'como', 'basada']
are ['\n\n\n', 'teniendo', 'tendrá', 'debe', 'acerca']
talk ['xxfake', 'hizo', 'von', 'descubierto', 'and']
which ['urbano', 'natural', 'artístico', 'poblacional', 'externos']
also ['local', 'junto', 'oficial', 'como', 'e']
has ['utilizan', 'deben', 'símbolo', 'permiten', 'tiene']
were ['local', 'teniendo', 'diciendo', 'encargado', 'cuya']
but ['natural', '\n\n\n', ':', 'externos', 'cuadrados']
have ['pertenecían', 'descubrimiento', 'utilizan', 'símbo

village ['vista', 'marzo', 'septiembre', 'febrero', 'noviembre']
art ['titulado', 'realizado', 'titulada', 'producido', 'lanzado']
player ['entre', 'y', 'sus', 'desde', 'natural']
final ['latinoamérica', 'españa', 'dios', 'estados', 'andalucía']
community ['grabar', 'lanzó', 'lanzado', 'crear', 'desarrollar']
again ['publicó', 'revista', 'incluyendo', 'ficción', 'cine']
began ['ángel', 'óscar', 'fin', 'haber', 'entonces']
army ['entre', 'cabe', 'estudió', 'nacido', 'educación']
award ['entonces', 'en', 'junto', 'como', 'hoy']
without ['geografía', 'sinopsis', 'administrativa', 'antigüedad', 'geográfica']
death ['publicado', 'productor', 'crítico', 'grabar', 'lanzado']
built ['y', 'esa', 'que', 'misma', 'o']
men ['ese', 'mayor', 'principal', 'estas', 'estos']
large ['obligado', 'detrás', 'aliados', 'dejando', 'delante']
site ['originaria', 'rica', 'civil', 'pública', 'tenía']
using ['medalla', 'policía', 'nunca', 'recepción', 'marina']
white ['allá', 'pasado', 'rico', 'salvador', 'escap

someone ['inaugurado', 'frank', 'edward', 'microsoft', 'escrita']
civil ['cuentan', 'sinopsis', 'permitir', 'duda', 'fundamentales']
police ['facultad', 'sudamérica', 'canal', 'vía', 'científico']
charles ['semana', 'comienzan', 'cuentan', 'presentan', 'buenos']
eastern ['santo', 'tribunal', 'les', 'santiago', 'cuentan']
body ['tenía', 'mexicano', 'sus', 'una', '²']
question ['posible', 'común', 'máximo', 'poner', 'través']
italian ['colombiano', 'venezolano', 'chileno', '’', '|']
featured ['autoridades', 'n', 'township', 'dictadura', 'papa']
week ['español', 'cómo', 'ese', 'qué', 'francés']
texas ['sus', 'sectores', 'fecha', 'ellos', 'mucha']
chief ['se', 'nos', 'descubre', 'duda', 'comienzan']
close ['julio', 'caso', 'momento', 'efecto', 'marzo']
match ['dijo', 'algo', 'tocar', 'escribir', 'américa']
roman ['windows', '>', 'corto', 'hotel', 'amarillo']
come ['un', 'griego', 'francés', 'español', 'dijo']
opened ['óscar', 'publicación', 'fin', 'contrario', 'n']
tour ['ese', 'áfrica', '

woman ['tenía', 'existen', 'metal', 'evidencia', 'pasado']
officer ['tenía', 'nació', 'estas', 'esas', 'nuestros']
greek ['femenino', 'chileno', 'juvenil', 'próximo', 'municipal']
recent ['personaje', 'actores', 'campus', 'investigación', 'busca']
sent ['escribió', 'n', 'haber', 'ángel', 'óscar']
speed ['²', 'vivo', 'josé', 'para', 'blanco']
money ['español', 'nacido', 'mexicano', 'catalán', 'esa']
saw ['\\', 'o', 'ni', 'roja', 'for']
selected ['ni', 'además', 'poco', 'tampoco', 'álvarez']
true ['richard', 'javier', 'departamento', 'alejandro', 'jaime']
regular ['trabajando', 'dedicado', 'realizando', 'aunque', 'denominado']
awarded ['ibérica', 'ni', 'oriental', 'menor', 'metropolitana']
commercial ['recibió', 'parroquia', 'ministros', 'autores', 'anunció']
cities ['febrero', 'marzo', 'hace', 'mayo', 'noviembre']
contains ['trama', 'derecha', 'depende', 'multitud', 'religiosa']
trade ['miss', 'presentar', 'nacido', 'raúl', 'actuar']
degree ['imposible', 'ampliamente', 'jones', 'mexican

understand ['patrón', 'líder', 'adicionales', 'punta', 'importantes']
episodes ['eso', 'ese', 'esa', 'dicha', 'cómo']
gives ['tendrá', 'tendría', 'muestra', 'podría', 'recibió']
educational ['extremo', 'particulares', 'estudiante', 'satélite', 'físico']
daily ['debajo', 'encima', 'trataba', 'latinoamérica', 'muestran']
williams ['fundamentalmente', 'sus', 'bastante', 'destacados', 'especialmente']
latin ['ee.uu', 'xxi', 'dvd', '#', 'presentes']
completely ['jugando', 'integrado', 'ganando', 'convirtiéndose', 'negó']
products ['artes', 'famoso', 'utilizar', 'iniciar', 'desarrollar']
dark ['bolivia', 'tratar', 'chicago', 'venezuela', 'adultos']
attention ['español', 'francés', 'hace', 'británico', 'castellano']
religion ['líder', 'equivalente', 'nacido', 'lanzado', 'jazz']
von ['domingo', 'francisco', 'agustín', 'viernes', 'juan']
mind ['todo', 'voz', 'petróleo', 'peso', 'toda']
administrative ['quito', 'xxfake', 'compuestos', 'crea', 'establecieron']
cut ['basada', 'sin', 'considerado',

piece ['ampliamente', 'demasiado', 'hidalgo', 'it', 'buscando']
growth ['comenzar', 'utilizar', 'resolver', 'continuar', '1906']
trust ['bienes', 'ideal', 'cuentos', 'socialista', 'grados']
drums ['profundidad', 'poesía', 'verdadero', 'boca', 'padre']
murder ['vista', 'on', 'convertirse', 'entrar', 'llevar']
normal ['hermana', 'tomando', 'causa', 'carl', 'chris']
twenty ['trabajar', 'dicho', 'seguro', 'mejorar', 'televisión']
commonly ['destaca', 'marcó', 'práctica', 'publicaciones', 'considera']
avoid ['nunca', 'luego', 'críticas', 'reparto', 'protestas']
tony ['comienzan', 'ojo', 'azul', 'espíritu', 'suprema']
norwegian ['escolar', 'ben', 'walter', 'park', 'incluido']
context ['cambia', 'altamente', 'aparentemente', 'provincial', 'comienza']
revolution ['numerosos', 'muchas', 'misma', '1965', 'casi']
fully ['perdiendo', 'convocado', 'casado', 'negociaciones', 'cuarenta']
wars ['1966', '1948', '1942', '1947', 'mediados']
leaves ['representante', 'hijo', 'mas', 'sexto', 'tercero']
dist

vietnam ['incluso', 'crecimiento', 'frecuencia', 'oxígeno', 'muchos']
dvd ['don', 'los', 'principios', 'julio', 'my']
refers ['principio', 'ángel', 'momento', 'óscar', '1943']
heat ['1952', 'claramente', 'famoso', '1880', 'revolucionario']
boat ['decidió', 'coronel', 'jorge', 'gobernador', 'emperador']
fell ['alcanzó', 'desde', 'escribió', 'ejerció', 'ficción']
arena ['supuso', 'aves', 'publicidad', 'pidió', '1904']
roads ['anterior', 'mismo', 'regreso', 'italiano', 'pasado']
core ['hacía', 'alcanza', 'posible', 'encontró', 'temprana']
kill ['reino', 'capítulo', 'estados', 'juana', 'alrededor']
athletic ['canales', 'fórmula', 'públicos', 'propuso', 'michael']
negative ['disponible', 'dada', 'aproximadamente', 'destaca', 'académico']
sixth ['sectores', 'señala', 'señaló', 'cádiz', 'universal']
edge ['et', 'in', 'township', 'y', 'e']
jesus ['james', 'pacientes', 'adicionales', 'muestras', 'posiblemente']
colonel ['corazón', 'peligro', 'cielo', 'duda', 'blanco']
chosen ['algunos', 'además

opposite ['notables', 'burgos', 'igualmente', 'david', 'sido']
streets ['fecha', 'habitualmente', 'adaptación', '1919', 'describió']
anna ['alexander', 'nieto', 'jiménez', 'palma', 'mayores']
extension ['hace', 'tenía', 'von', 'regresa', 'aceptó']
shooting ['tema', 'informó', 'consideró', 'llegó', 'declaró']
ready ['medios', 'jean', 'grandes', 'veinte', 'críticos']
pick ['moral', 'río', 'biología', 'onda', 'visión']
composition ['nos', 'quiere', 'vista', 'simplemente', 'sí']
lands ['sexto', '1977', 'novela', 'actuó', '1953']
interests ['creciente', '1915', 'clásicos', 'habla', 'castilla']
closing ['cambió', 'corto', 'corriente', 'religioso', 'imperial']
retirement ['raíces', 'embajador', 'rodrigo', 'definitivamente', 'lanzado']
scientists ['clásicos', 'diversas', '1914', 'debutó', '1971']
techniques ['renta', 'grabó', 'danza', 'importantes', 'democrático']
participate ['finalizar', 'fe', 's.', 'mes', 'm.']
anniversary ['obligado', 'pública', 'hace', '1973', 'garcía']
discovery ['morale

approval ['católico', 'maria', 'capital', 'constitucional', 'regresa']
horses ['suficiente', '1942', 'entrar', 'anterior', 'republicano']
visiting ['felipe', 'salamanca', 'raúl', 'técnicas', 'miami']
supporters ['1904', 'empieza', 'grabó', 'encontrar', 'quería']
pink ['cuarto', 'perdido', 'street', 'perder', 'ropa']
roots ['fueron', 'evolución', 'flores', 'hacía', 'ramírez']
premiered ['artista', 'haber', 'protagonista', 'totalidad', 'papa']
adults ['young', 'totalmente', 'xxfake', 'asociación', 'niña']
immediate ['técnicos', 'religioso', 'procedentes', 'provenientes', 'previamente']
strip ['casi', 'bastante', 'fácilmente', '1908', 'nuevos']
bound ['química', 'dos', 'viajó', 'estudios', 'aparecen']
hull ['mexicana', 'frente', 'xxfake', 'crearon', 'aceptó']
origins ['fuertemente', 'tomar', 'materia', 'deja', 'necesaria']
tie ['pequeñas', 'gas', 'realizar', 'recientes', 'dólares']
liberty ['xxpad', 'dúo', 'trinidad', 'vio', 'peces']
siege ['1917', 'di', '1914', 'vive', 'rodríguez']
portr

0.037943696450428395

In [ ]:
def top_n_translation_acc(top_n=1):
    p = learn.model[0].encoder.p
    learned_embeddings = learn.model[0].encoder.p @ learn.model[0].encoder.weight
    
    hits = 0
    for k, v in en_es_dict.items():
        idx_en = vocab_en.index(k)
        diff = learn.model[0].encoder.weight - learned_embeddings[idx_en]
        candidates = diff.sum(-1).argsort(descending=False)[:top_n]
        
        for vv in v:
            idx_es = vocab_es.index(vv)
            if idx_es in candidates:
                hits += 1
                break
    return hits/len(en_es_dict)

In [56]:
top_n_translation_acc(100)

0.025703794369645042